# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [38]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

In [39]:
df=pd.read_csv('users_behavior.csv')

In [40]:
df.is_ultra.value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

In [41]:
df.sample(10)

,calls,minutes,messages,mb_used,is_ultra
55,13.0,106.03,16.0,37328.45,1
670,33.0,241.75,9.0,23708.50,0
586,6.0,51.08,16.0,20634.98,1
979,34.0,268.50,10.0,8859.08,0
2490,84.0,580.08,130.0,24932.59,1
336,11.0,70.50,0.0,2042.74,0
882,66.0,494.47,0.0,17360.07,0
2067,48.0,344.00,29.0,5575.64,0
2644,48.0,436.16,13.0,29072.01,0
2044,83.0,529.53,93.0,13796.44,0


Открыли файл, проверили на пропуски, дубликаты - все в порядке. Абонентов с тарифом Ультра всего около трети. Таким образом классы не очень сбалансированы. 

## Разбейте данные на выборки

In [42]:
features = df.drop('is_ultra', axis = 1) # извлеките признаки 
target = df['is_ultra'] # извлеките целевой признак
features_train, features_tmp, target_train, target_tmp = train_test_split(
   features, target, test_size=0.4, random_state=12345) 
features_valid, features_test, target_valid, target_test = train_test_split(
   features_tmp, target_tmp, test_size=0.5, random_state=12345) 

In [43]:
features_train.shape

(1928, 4)

In [44]:
features_valid.shape

(643, 4)

In [45]:
features_test.shape

(643, 4)

In [46]:
target_train.shape

(1928,)

In [47]:
target_valid.shape

(643,)

In [48]:
target_test.shape

(643,)

Разбили всю выборку на 3 категории: тестовая выборка - 60%, валидация - 20 %, тестовая - 20 %.

## Исследуйте модели

### DecisionTreeClassifier

In [49]:
best_model_dtc = None
best_result_dtc = 0
for leaf in range(2, 100):
    for depth in range(1, 15):
        model = DecisionTreeClassifier(random_state=12345, max_depth=depth, max_leaf_nodes=leaf,class_weight ='balanced') # обучение модели с заданной глубиной дерева
        model.fit(features_train,target_train) # обучение  модели
        predictions = model.predict(features_valid) # предсказания 
        result = accuracy_score(target_valid, predictions) #  качество модели
        if result > best_result_dtc:
            best_model_dtc = model # сохранили наилучшую модель
            best_result_dtc = result#сохранили наилучшее значение метрики accuracy на валидационных данных
            i = depth #оптимальная глубина дерева
            max_leaf=leaf
        

In [50]:
print("Accuracy лучшей модели:", best_result_dtc)
print("Максимальная глубина дерава решений, обеспечивающая лучшиее предсказание по критерию Accuracy:", i)
print("максимальное количество конечных узлов, обеспечивающая лучшиее предсказание по критерию Accuracy:", max_leaf)

Accuracy лучшей модели: 0.7838258164852255
Максимальная глубина дерава решений, обеспечивающая лучшиее предсказание по критерию Accuracy: 2
максимальное количество конечных узлов, обеспечивающая лучшиее предсказание по критерию Accuracy: 3


- max_depth максимальная глубина дерева решений. Максимальная глубина дерева решений не может быть введена по умолчанию.Если она не введена, дерево решений не будет ограничивать глубину поддерева при построении поддерева. Вообще говоря, это значение можно игнорировать, если данных или функций мало. Если модель имеет большой размер выборки и много функций, рекомендуется ограничить эту максимальную глубину.Конкретное значение зависит от распределения данных. Обычно используемые значения могут находиться в диапазоне 10–100.
- random_state случайное состояние - фиксирует псевдослучайность для алгоритма обучения. Если укажете random_state=None(по умолчанию), то псевдослучайность будет разной всегда.
- max_leaf_nodes максимальное количество конечных узлов  - ограничивая максимальное количество конечных узлов, можно предотвратить переобучение. По умолчанию установлено «None», то есть максимальное количество конечных узлов не ограничено. Если добавлено ограничение, алгоритм построит оптимальное дерево решений в пределах максимального количества конечных узлов. 

### RandomForestClassifier

In [51]:
best_model_rnd = None
best_result_rnd = 0
for est in range(1, 30): #задаем количество деревьев в лесу
    model = RandomForestClassifier(random_state=12345, n_estimators=est, class_weight ='balanced') 
    model.fit(features_train, target_train) 
    predictions = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions)
    if result > best_result_rnd:
        best_model_rnd = model # сохранили наилучшую модель
        best_result_rnd = result #сохранили наилучшее значение метрики accuracy на валидационных данных
   

In [52]:
print("Accuracy лучшей модели:", best_result_rnd)

Accuracy лучшей модели: 0.7947122861586314


### LogisticRegression

In [62]:
model_log = LogisticRegression(random_state=12345) 
model_log.fit(features_train, target_train) # обучите модель на тренировочной выборке
result = model_log.score(features_valid, target_valid) # получите метрику качества модели на валидационной выборке

print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.7589424572317263


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### Вывод

Обучили три модели. Точность на выборке валидации у всех сопоставима и выше требуемого уровня в 0.75. 
- Для метода деревьев решения точность составила 0.79 и оптимальная глубина дерева - 9, максимальное количество конечных узлов, обеспечивающая лучшиее предсказание по критерию Accuracy- 79
- Для метода случайных деревьев точность - 0.795;
- Для логистической регрессии точность - 0.759.
- Сохранили все модели для дальнейшего исследования. 

## Проверьте модель на тестовой выборке

In [54]:
predictions_dtc = best_model_dtc.predict(features_test) # получите предсказания модели
result_dtc = accuracy_score(target_test, predictions_dtc) # посчитайте качество модели на тестовой выборке
print("Accuracy DecisionTreeClassifier:",result_dtc)

predictions_rnd = best_model_rnd.predict(features_test)
result_rnd = accuracy_score(target_test, predictions_rnd)
print("Accuracy RandomForestClassifier:" ,result_rnd)

predictions_log = model_log.predict(features_test)
result_log = accuracy_score(target_test, predictions_log)
print("Accuracy LogisticRegression:",result_log)


Accuracy DecisionTreeClassifier: 0.7698289269051322
Accuracy RandomForestClassifier: 0.7853810264385692
Accuracy LogisticRegression: 0.7402799377916018


На тестовой выборке точность немного упала, но не так сильно.
- Для метода деревьев решения точность составила 0.785;
- Для метода случайных деревьев точность - 0.781;
- Для логистической регрессии точность - 0.74.


**Проверим можно ли улучшить результат за счет голосования моделей**

In [55]:
predictions_sum = pd.DataFrame(predictions_dtc+predictions_rnd+predictions_log)

In [56]:
predictions_sum.columns=['value']

In [57]:
predictions_sum['value'] = predictions_sum['value'].replace([1,2,3], [0,1,1])
result_sum = accuracy_score(target_test, predictions_sum)
print("Accuracy голосования моделей:",result_log)

Accuracy голосования моделей: 0.7402799377916018


Проверили голосование - результат стал хуже лучшей модели. Эксперимент признан неудачным)

### Вывод

На основе тестовой выборки проверили модели. Победил алгоритм классификации решающее дерево с результатом точности (accuracy) 0.785

## (бонус) Проверьте модели на адекватность

In [58]:
report = classification_report(target_test, predictions_dtc, target_names=['Smart', 'Ultra'])
print(report)


              precision    recall  f1-score   support

       Smart       0.78      0.92      0.85       440
       Ultra       0.72      0.44      0.55       203

    accuracy                           0.77       643
   macro avg       0.75      0.68      0.70       643
weighted avg       0.76      0.77      0.75       643



In [59]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_train,target_train)
predictions_dmm = dummy_clf.predict(features_test)
dummy_clf.score(features_test, target_test)

0.6842923794712286

In [60]:
report = classification_report(target_test, predictions_dmm, target_names=['Smart', 'Ultra'])
print(report)

              precision    recall  f1-score   support

       Smart       0.68      1.00      0.81       440
       Ultra       0.00      0.00      0.00       203

    accuracy                           0.68       643
   macro avg       0.34      0.50      0.41       643
weighted avg       0.47      0.68      0.56       643



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


- DummyClassifier — это примитивный классификатор, который делает прогнозы с использованием простых правил. Этот классификатор полезен как простая базовая линия для сравнения с другими (реальными) классификаторами. Не используется для реальных задач.
- Для метода деревьев решения точность составила 0.785, а для примитивного классификатора лучший результат при использовании разных стратегий прогнозирования - 0.684.
- При сравнении различных критериев для примитивного классификатора видно, что его стратегия - игнорирование меньшего по величине класса. Хотя для дерева решений второй класс (тариф Ультра) тоже "пострадал", но в разработанного алгоритме нет такого провала по второму классу выборки. 
- Таким образом, можно сказать, что разработанный классификатор существо лучше простых стратегий решения.

### Вывод

Из представленной таблицы можно сделать следующие выводы:
- precision (точность) по тарифу Смарт гораздо выше и составляет 0.8, при этом общем количестве значений для тарифа смарт в тестовой выборке составляет 440;
- precision (точность) по тарифу Ультра гораздо ниже и составляет 0.73, при этом общем количестве значений для тарифа смарт в тестовой выборке составляет 203;
- в виду неравномерности классов accuracy (точность) точность составляет 0.785;
- recall (полнота) по тарифу Смарт гораздо выше и составляет 0.91;
- recall (полнота) по тарифу Ультра существенно ниже и составляет 0.51;
- F-мера (в общем случае $\ F_\beta$) — среднее гармоническое precision и recall для тарифов Смарт и Ультра также существенно отличается и составляет 0.85, 0.6 соответственно; 
- Для метода деревьев решения точность составила 0.785, а для примитивного классификатора лучший результат при использовании разных стратегий прогнозирования - 0.684.При сравнении различных критериев для примитивного классификатора видно, что его стратегия - игнорирование меньшего по величине класса. Хотя для дерева решений второй класс (тариф Ультра) тоже "пострадал", но в разработанного алгоритме нет такого провала по второму классу выборки. 
- Таким образом, можно сказать, что разработанный классификатор существо лучше простых стратегий решения.
- таким образом, представленный алгоритм гораздо лучше работает для тарифа Смарт что обусловлено перекосом в данных.



## Общий вывод

- Разбили всю выборку на 3 категории: тестовая выборка - 60%, валидация - 20 %, тестовая - 20 %.
- Обучили три модели. Точность на выборке валидации у всех сопоставима и выше требуемого уровня в 0.75. Для метода решающего дерева точность составила 0.79 и оптимальная глубина дерева - 9, максимальное количество конечных узлов, обеспечивающая лучшее предсказание по критерию Accuracy- 79. Для метода случайных деревьев точность - 0.795. Для логистической регрессии точность - 0.759.
- Выбрали модель с максимально большим значением *accuracy*. Требуемое значение -   0.75. При проверке *accuracy* на тестовой выборке получено значение 0.785. На тестовой выборке точность немного упала, но не так сильно. Для метода деревьев решения точность составила 0.785, для метода случайных деревьев точность - 0.781, для логистической регрессии точность - 0.74.
- precision (точность) по тарифу Смарт гораздо выше и составляет 0.8, при этом общем количестве значений для тарифа смарт в тестовой выборке составляет 440; precision (точность) по тарифу Ультра гораздо ниже и составляет 0.73, при этом общем количестве значений для тарифа смарт в тестовой выборке составляет 203;
- в виду неравномерности классов accuracy (точность) точность составляет 0.785;
- recall (полнота) по тарифу Смарт гораздо выше и составляет 0.91; recall (полнота) по тарифу Ультра существенно ниже и составляет 0.51;
- F-мера (в общем случае $\ F_\beta$) — среднее гармоническое precision и recall для тарифов Смарт и Ультра также существенно отличается и составляет 0.85, 0.6 соответственно; 
- Для метода деревьев решения точность составила 0.785, а для примитивного классификатора лучший результат при использовании разных стратегий прогнозирования - 0.684.При сравнении различных критериев для примитивного классификатора видно, что его стратегия - игнорирование меньшего по величине класса. Хотя для дерева решений второй класс (тариф Ультра) тоже "пострадал", но в разработанного алгоритме нет такого провала по второму классу выборки. 
- таким образом, представленный алгоритм гораздо лучше работает для тарифа Смарт что обусловлено перекосом в данных.
- необходимо отметить, что метод случайных деревьев немного выиграл на выборке валидации, но немного уступил на тестовой выборке, что может быть случайностью.
